In [67]:
import pandas as pd
import collections
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import scipy.stats as st
from tqdm import tqdm
import itertools
from numpy import exp
from statistics import mean
import scipy.stats as stats
import pickle
from typing import OrderedDict

In [15]:
def manipulate_data(raw, marker):
    '''anchor data header to the marker given'''
    df = raw
    df.drop([1,2], inplace = True)
    subject = [i[0:5] for i in df.columns]
    subject[0] = "subject"
    df.loc[-1] = subject
    df = df.sort_index()
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace = True)
    df.rename(columns = {df.columns[0] : "video"}, inplace = True)
    df["video"][1] = "markers"
    df["video"][2] = "axis"
    df_transposed = df.T
    df_transposed.columns = df_transposed.iloc[0]
    df_transposed.drop('video', axis = 0, inplace = True)
    marker_list = df_transposed["markers"].unique()
    subject_list = df_transposed["subject"].unique()
    DICT_repetition = collections.defaultdict()
    for subject in subject_list:
        repetition = int(sum((df_transposed["markers"] == marker) & (df_transposed["subject"] == subject)) / 3)
        DICT_repetition[subject] = repetition
    
    standard_marker_list = ['C7', 'LACR', 'LASIS', 'LCAP', 'LFA1', 'LFA2', 'LFA3', 'LHEAD',
        'LHLE', 'LHMC1', 'LHMC2', 'LHMC3', 'LHME', 'LICR', 'LPSIS', 'LRSP',
        'LTEMP', 'LUA1', 'LUA2', 'LUA3', 'LUA4', 'LUSP', 'RACR', 'RASIS',
        'RCAP', 'RFA1', 'RFA2', 'RFA3', 'RHEAD', 'RHLE', 'RHMC1', 'RHMC2',
        'RHMC3', 'RHME', 'RICR', 'RPSIS', 'RRSP', 'RTEMP', 'RUA1', 'RUA2',
        'RUA3', 'RUA4', 'RUSP', 'STER', 'T10', 'T4', 'T7', 'XPRO',
        'LMIDDLE', 'LTHUMB', 'RMIDDLE', 'RTHUMB',
        'Pelvis_JOINT_ACCELERATION', 'Pelvis_JOINT_ANGLE', 'T8',
        'L_Elbow_JOINT_ACCELERATION', 'L_Elbow_JOINT_ANGLE',
        'L_Shoulder_JOINT_ACCELERATION', 'L_Shoulder_JOINT_ANGLE',
        'L_Wrist_JOINT_ACCELERATION', 'L_Wrist_JOINT_ANGLE',
        'Neck_JOINT_ACCELERATION', 'Neck_JOINT_ANGLE',
        'R_Elbow_JOINT_ACCELERATION', 'R_Elbow_JOINT_ANGLE',
        'R_Shoulder_JOINT_ACCELERATION', 'R_Shoulder_JOINT_ANGLE',
        'R_Wrist_JOINT_ACCELERATION', 'R_Wrist_JOINT_ANGLE',
        'Waist_JOINT_ACCELERATION', 'Waist_JOINT_ANGLE']
    discard_index_list = []
    for i in marker_list:
        if i not in standard_marker_list:
            discard_index_list.extend(df_transposed[df_transposed['markers'] == i].index)
    df_transposed.drop(discard_index_list, axis = 0, inplace = True)
    
    subject_marker_dict = collections.defaultdict()
    repetition_list = []
    for subject in subject_list:
        subject_marker_list = df_transposed[df_transposed["subject"] == subject]["markers"].unique()
        subject_marker_dict[subject] = subject_marker_list
        for i in range(len(subject_marker_list)):
            for j in range(DICT_repetition[subject]):
                repetition_list.extend([j + 1, j + 1, j + 1])
    df_transposed.insert(3, 'repetition', repetition_list)



    return df_transposed, subject_list, subject_marker_dict, DICT_repetition


In [16]:
def get_coordinates_dict(data, data_point, point_to_analyse, subject_list, repetition_dict, subject_marker_dict):
    ''' 
    first output: separate out the coordinates for each repetition for each subject;
    second output: calculate out the mean trajectory;
    third output: total number of repetitions considered
    (not yet in use)
    '''
    # separate out the coordinates for each repetition for each subject
    subject_coordinates_repetition_dict = collections.defaultdict()
    coordinates_array = None
    n_sample = 0
    for subject in subject_list:
        # only if the subject has the point to analyse, otherwise, skipped
        if point_to_analyse in subject_marker_dict[subject]:
            n_sample = n_sample + repetition_dict[subject]

            for i in range(repetition_dict[subject]):
                x_data, y_data = np.array(data[(data["subject"] == subject) & (data["markers"] == point_to_analyse) & (data["axis"] == "X") & (data["repetition"] == i + 1)].drop(columns = ["subject", "markers", "repetition", "axis"]).T).astype(float), np.array(data[(data["subject"] == subject) & (data["markers"] == point_to_analyse) & (data["axis"] == "Y") & (data["repetition"] == i + 1)].drop(columns = ["subject", "markers", "repetition", "axis"]).T).astype(float)
                a, b = x_data[0], y_data[0]
                x_data = x_data - a
                y_data = y_data - b
                coordinates = np.concatenate((x_data, y_data), axis = 1)
                if coordinates_array is None:
                    coordinates_array = coordinates
                elif  coordinates_array is not None:
                    coordinates_array = np.append(coordinates_array, coordinates)
            
            # compile into each repetition for each subject
            subject_coordinates_repetition_dict[subject] = coordinates_array.reshape(-1, data_point,2)
    
    # calculate the trajectory mean
    coordinates_mean = np.mean(np.concatenate(list(subject_coordinates_repetition_dict.values())), axis = 0)

    return subject_coordinates_repetition_dict, coordinates_mean, n_sample

In [17]:
def get_joint_angle_dict(data, data_point, JA_to_analyse, axis_to_analyse, subject_list, repetition_dict, subject_marker_dict):
    ''' 
    first output: separate out the JA for each repetition for each subject;
    second output: calculate out the mean JA trajectory;
    third output: total number of repetitions considered
    '''
    # separate out the JA for each repetition for each subject
    subject_JA_repetition_dict = collections.defaultdict()
    JA_array = None
    n_sample = 0
    for subject in subject_list:
        if JA_to_analyse in subject_marker_dict[subject]:
            n_sample = n_sample + repetition_dict[subject]

            for i in range(repetition_dict[subject]):
                JA = np.array(data[(data["subject"] == subject) & (data["markers"] == JA_to_analyse) & (data["axis"] == axis_to_analyse) & (data["repetition"] == i + 1)].drop(columns = ["subject", "markers", "repetition", "axis"]).T).astype(float)
                # for x-coordinates(time)
                time = np.array([ int(i) for i in range(JA.shape[0])]).reshape(JA.shape[0],-1)
                time_series_coordinates = np.concatenate((time, JA), axis = 1)
                if JA_array is None:
                    JA_array = time_series_coordinates
                elif  JA_array is not None:
                    JA_array = np.append(JA_array, time_series_coordinates)
                
            # compile into each repetition for each subject
            subject_JA_repetition_dict[subject] = JA_array.reshape(-1, data_point, 2)

    # calculate the trajectory mean
    JA_mean = np.mean(np.concatenate(list(subject_JA_repetition_dict.values())), axis = 0)

    return subject_JA_repetition_dict, JA_mean, n_sample


In [18]:
def generate_norm_dist(normal_dist_data, show_graph = False):
    ''' to generate normal distribution for each point to analyse and return the confidence interval of 95%, show graph if needed'''
    confidence = st.t.interval(alpha=0.95, df=len(normal_dist_data)-1, loc=np.mean(normal_dist_data), scale=st.sem(normal_dist_data))
    if show_graph:
        count, bins, ignored = plt.hist(list(normal_dist_data), len(normal_dist_data))
        plt.show()
    return confidence

def generate_score(normative_mean, stroke_mean, plot_graph = False):
    '''calculate dtw score between two mean graph, plot graph if needed to see the time warping relation'''
    score = dtw.distance(normative_mean[:,1], stroke_mean[:,1])
    if plot_graph:
        fig, axs = plt.subplots(nrows=2, ncols=1, sharex='all', sharey='all')
        path = dtw.warping_path(normative_mean[:,1], stroke_mean[:,1])
        dtwvis.plot_warping(normative_mean[:,1], stroke_mean[:,1], path, fig = fig, axs = axs)
        plt.show()
    return score

In [19]:
def get_feature_dict(analyse_dict, axis_list):
    feature_dict = collections.defaultdict()
    for key in analyse_dict.keys():
        try:
            for i in analyse_dict[key]:
                feature_dict[i] = axis_list[key]
        except:
            continue
    return feature_dict

In [21]:
ROM_dict = collections.defaultdict()
ROM_dict[('elbow', 'X')] = tuple((0,140))
ROM_dict['elbow', 'Z'] = tuple((-80, 80))
ROM_dict[('shoulder', 'X')] = tuple((-50, 180))
ROM_dict[('shoulder', 'Y')] = tuple((-180, 50))
ROM_dict[('shoulder', 'Z')]= tuple((-90, 90))
ROM_dict[('wrist', 'X')] = tuple((-60, 60))
# ROM_dict[('wrist', 'Z')] = tuple((-20, 20))
data_point = 300

# inputs
TOWEL_vertFold = {"shoulder":["X"], "elbow":["X"], "wrist":["X"]}
TOWEL_horiFold = {"shoulder":["X", "Y"], "elbow":["X"], "wrist":["X"]}
GRASP = {"shoulder":["X"], "elbow":["X"], "wrist":["X"]}
LATERAL = {"shoulder":["X", "Y"], "elbow":["X"]}
MOUTH = {"shoulder":["X", "Y"], "elbow":["X", "Z"], "wrist":["X"]} 
KEY = {"shoulder":["X", "Y"], "elbow":["X", "Z"]}

analyse_dict_L = {"shoulder": ["L_Shoulder_JOINT_ANGLE"], "elbow": ["L_Elbow_JOINT_ANGLE"], "wrist": ["L_Wrist_JOINT_ANGLE"]}
analyse_dict_R = {"shoulder": ["R_Shoulder_JOINT_ANGLE"], "elbow": ["R_Elbow_JOINT_ANGLE"], "wrist": ["R_Wrist_JOINT_ANGLE"]}



In [28]:
normative_towel_vert_R = pd.read_csv(r"Z:\DataCollection\BCISoftRoboticsGloveIntervention\Stroke\Miscellaneous\BCI001\Normative\1. EXPORTS\EXPORT_towel\RightJA_2Pick1_Bwd.txt", sep = '\t')
stroke_towel_vert_R = pd.read_csv(r"C:\Users\Administrator\Documents\gitHub\BCI\data\stroke_towel_vert_R", sep = '\t')

normative_towel_hori_R = pd.read_csv(r"Z:\DataCollection\BCISoftRoboticsGloveIntervention\Stroke\Miscellaneous\BCI001\Normative\1. EXPORTS\EXPORT_towel\RightJA_4Pick2_Cross.txt", sep = '\t')
stroke_towel_hori_R = pd.read_csv(r"C:\Users\Administrator\Documents\gitHub\BCI\data\stroke_towel_hori_R", sep = '\t')

normative_grasp_R = pd.read_csv(r"Z:\DataCollection\BCISoftRoboticsGloveIntervention\Stroke\Miscellaneous\BCI001\Normative\1. EXPORTS\EXPORT_grasp\RightJA_2Block_Shelf.txt", sep = '\t')
stroke_grasp_R = pd.read_csv(r"C:\Users\Administrator\Documents\gitHub\BCI\data\stroke_grasp_R", sep = '\t')

normative_lateral_R = pd.read_csv(r"Z:\DataCollection\BCISoftRoboticsGloveIntervention\Stroke\Miscellaneous\BCI001\Normative\1. EXPORTS\EXPORT_lateral\RightJA_1Start_Cross.txt", sep = '\t')
stroke_lateral_R = pd.read_csv(r"C:\Users\Administrator\Documents\gitHub\BCI\data\stroke_lateral_R", sep = '\t')

normative_mouth_R = pd.read_csv(r"Z:\DataCollection\BCISoftRoboticsGloveIntervention\Stroke\Miscellaneous\BCI001\Normative\1. EXPORTS\EXPORT_mouth\RightJA_1Start_Mouth.txt", sep = '\t') # input
stroke_mouth_R = pd.read_csv(r"C:\Users\Administrator\Documents\gitHub\BCI\data\stroke_mouth_R", sep = '\t')


normative_key_R = pd.read_csv(r"Z:\DataCollection\BCISoftRoboticsGloveIntervention\Stroke\Miscellaneous\BCI001\Normative\1. EXPORTS\EXPORT_keystand\RightJA_2Touchkey_Turn.txt", sep = '\t')
stroke_key_R = pd.read_csv(r"C:\Users\Administrator\Documents\gitHub\BCI\data\stroke_key_R", sep = '\t')

FileNotFoundError: [Errno 2] No such file or directory: 'Z:\\DataCollection\\BCISoftRoboticsGloveIntervention\\Stroke\\Miscellaneous\\BCI001\\Normative\\1. EXPORTS\\EXPORT_towel\\RightJA_2Pick1_Bwd.txt'

In [105]:
def upper_threshold(result_df):
    result_list = []
    for index in range(len(result_df)):
        if (int(result_df["Stroke Patient Score"][index]) > result_df["Normative Confidence Interval"][index][0]) == True and (int(result_df["Stroke Patient Score"][index]) < result_df["Normative Confidence Interval"][index][1]) == True:
            # result_df["Drop (Upper threshold)"][index] = 1
            result_list.append(1)
        else:
            # result_df["Drop (Upper threshold)"][index] = 0
            result_list.append(0)

    result_df['Drop (Upper threshold)'] = result_list
    return result_df

def get_trial_ROM(df, feature_dict, first_result):
    df['feature'] = np.array(df["markers"] + '_'+ df["axis"])
    features_list = [i+'_'+j for i, k in feature_dict.items() for j in k]
    relevant_df = df[df["feature"].isin(features_list)]
    trial_rom_list = []
    for i in range(len(relevant_df)):
        trial_rom_list.append(max([float(k) for k in relevant_df.iloc[i,4:-1]]) - min([float(k) for k in relevant_df.iloc[i,4:-1]]))
        # print(trial_ROM)
    # test.iloc[:,4:-1]
    relevant_df["trial ROM"] = trial_rom_list
    average_list = collections.defaultdict()
    for feature in features_list:
        average_list[feature] = mean(relevant_df["trial ROM"][relevant_df["feature"] == feature])
    resorted_average_list = []
    for i in range(len(first_result)):
        if first_result["Feature"][i] == "Flexion/Extension":
            axis = "X"
        if first_result["Feature"][i] == "Abduction/Adduction":
            axis = "Y"
        if first_result["Feature"][i] == "External/Internal Rotation":
            axis = "Z"
        resorted_average_list.append(average_list[first_result["Joint Angle"][i] + '_' + axis])
    first_result["Trial ROM"] = resorted_average_list
    return relevant_df, features_list, first_result
    return df

def get_significance_vector(result_table, feature_dict, analyse_dict, ROM_dict):
    significancy_list = []
    rom_list = []
    total_calculation = []
    result_table = result_table.reset_index()
    for index in range(len(result_table)):
        for each_axis in feature_dict[result_table["Joint Angle"][index]]:
            for i, k in analyse_dict.items():
                if result_table["Joint Angle"][index] in k:
                    rangeOfMotion = tuple((i, each_axis))
                    break
        rom_list.append(ROM_dict[rangeOfMotion])
        if int(result_table["Drop (Upper threshold)"][index]) == 0:
            significancy = (result_table["Trial ROM"][index]) / (int(ROM_dict[rangeOfMotion][1]) - int(ROM_dict[rangeOfMotion][0]))
            significancy_list.append(float(significancy))
        else:
            significancy_list.append(0)
    # print(significancy_list)
    result_table["ROM"] = rom_list
    result_table["weight"] = significancy_list

    return result_table

In [23]:
def main(normative_df, stroke_df, task, analyse_dict, ROM_dict, data_point, name):
    data, subject_list, subject_marker_dict, repetition_dict = manipulate_data(normative_df, "L_Elbow_JOINT_ANGLE")
    stroke_data, stroke_list, stroke_marker_dict, stroke_repetition_dict = manipulate_data(stroke_df, "L_Elbow_JOINT_ANGLE")

    print("Starting for Task:", name)
    total_index = 0
    for key in task.keys():
        total_index = total_index + len(task[key])*len(analyse_dict[key])


    # consolidate end results
    feature_dict = get_feature_dict(analyse_dict, task)
    result = pd.DataFrame(columns = ['Joint Angle', 'Feature', 'Stroke Patient Score', 'Normative Confidence Interval'], index = [i for i in range(total_index)])
    rotation = {"X":"Flexion/Extension", "Y":"Abduction/Adduction", "Z":"External/Internal Rotation"}
    current_index = -1


    normal_dist_data = collections.defaultdict()

    feature_dict = get_feature_dict(analyse_dict, task)
    try:
        for each_feature in feature_dict.keys():
            print("joint angle:", each_feature)
            for each_axis in feature_dict[each_feature]:
                print("axis:", each_axis)
                current_index += 1
                analysing = rotation[each_axis]

                # do bootstraping to get normal distribution, confidence interval, dtw score for each point to analyse and each axis
                normative_JA, normative_JA_mean, normative_n_sample = get_joint_angle_dict(data, data_point, each_feature, each_axis, subject_list, repetition_dict, subject_marker_dict)
                stroke_JA, stroke_JA_mean, stroke_n_sample = get_joint_angle_dict(stroke_data, data_point, each_feature, each_axis, stroke_list, stroke_repetition_dict, stroke_marker_dict)
                normal_dist_list = []
                for subject in tqdm(subject_list):
                    individual_subject_list = [subject]
                    bootstrap_subject_list = [ x for x in subject_list if x != subject]
                    bootstrap_JA, bootstrap_JA_mean, bootstrap_n_sample = get_joint_angle_dict(data, data_point, each_feature, each_axis, bootstrap_subject_list, repetition_dict, subject_marker_dict)
                    individual_JA, individual_JA_mean, individual_n_sample = get_joint_angle_dict(data, data_point, each_feature, each_axis, individual_subject_list, repetition_dict, subject_marker_dict)
                    individual_score = generate_score(bootstrap_JA_mean, individual_JA_mean)
                    normal_dist_list.append(float(individual_score))
                confidence_interval = generate_norm_dist(normal_dist_list, show_graph=True)
                stroke_score = generate_score(normative_JA_mean, stroke_JA_mean, plot_graph = True)

                each_row = {}
                each_row["Joint Angle"] = each_feature
                each_row["Feature"] = analysing
                each_row["Stroke Patient Score"] = stroke_score
                each_row["Normative Confidence Interval"] = confidence_interval
                normal_dist_data['normative_'+str(each_feature) + '_' + str(each_axis)] = normal_dist_list
                normal_dist_data['stroke_'+str(each_feature)+'_'+str(each_axis)] = stroke_score

                sd = np.std(np.array(normal_dist_data['normative_'+str(each_feature) + '_' + str(each_axis)]))
                each_row["std of normative"] = sd

                mean = np.mean(np.array(normal_dist_data['normative_'+str(each_feature) + '_' + str(each_axis)]))
                each_row["mean of normative"] = mean

                temp_list = normal_dist_data['normative_'+str(each_feature) + '_' + str(each_axis)] + [normal_dist_data['stroke_'+str(each_feature)+'_'+str(each_axis)]]
                zscore_list = stats.zscore(temp_list)
                each_row["stroke z-score"] = zscore_list[-1]
                result.loc[current_index] = pd.Series(each_row)

        return data, result, normal_dist_data

    except:
        print("error at running the main calculation")
        data = None
        result =  None
        normal_dist_data = None
        return data, result, normal_dist_data


In [27]:
towelvert_R_df, result_towel_vert_R, norm_dist_towel_vert_R = main(normative_towel_vert_R, stroke_towel_vert_R, TOWEL_vertFold, analyse_dict_R, ROM_dict, data_point, 'towel vert R')

towelhori_R_df, result_towel_hori_R, norm_dist_towel_hori_R = main(normative_towel_hori_R, stroke_towel_hori_R, TOWEL_horiFold, analyse_dict_R, ROM_dict, data_point, 'towel hori R')

grasp_R_df, result_grasp_R, norm_dist_grasp_R = main(normative_grasp_R, stroke_grasp_R, GRASP, analyse_dict_R, ROM_dict, data_point, 'grasp R')

lateral_R_df, result_lateral_R, norm_dist_lateral_R = main(normative_lateral_R, stroke_lateral_R, LATERAL, analyse_dict_R, ROM_dict, data_point, 'lateral R')

mouth_R_df, result_mouth_R, norm_dist_mouth_R = main(normative_mouth_R, stroke_mouth_R, MOUTH, analyse_dict_R, ROM_dict, data_point, 'mouth R')

key_R_df, result_key_R, norm_dist_key_R = main(normative_key_R, stroke_key_R, KEY, analyse_dict_R, ROM_dict, data_point, 'key R')

NameError: name 'normative_towel_vert_R' is not defined

In [61]:
def upper_threshold(result_df):
    upper_threshold_list = []
    for index in range(len(result_df)):
        if (int(result_df["Stroke Patient Score"][index]) > result_df["Normative Confidence Interval"][index][0]) == True and (int(result_df["Stroke Patient Score"][index]) < result_df["Normative Confidence Interval"][index][1]) == True:
            upper_threshold_list.append(1)
            # result_df["Drop (Upper threshold)"][index] = 1
        else:
            upper_threshold_list.append(0)
            # result_df["Drop (Upper threshold)"][index] = 0
    result_df["Drop (Upper threshold)"] = upper_threshold_list
    return result_df

def get_norm_dist(results, normal_dist_data):
    mean_list = []
    sd_list = []
    zscore_list = []
    for index in range(len(results)):
        if results["Feature"][index] == "Flexion/Extension":
            axis = "X"
        elif results["Feature"][index] == "Abduction/Adduction":
            axis = "Y"
        elif results["Feature"][index] == "External/Internal Rotation":
            axis = "Z"
            
        mean = np.mean(np.array(normal_dist_data['normative_'+str(results["Joint Angle"][index]) + '_' + str(axis)]))
        mean_list.append(mean)
        sd = np.std(np.array(normal_dist_data['normative_'+str(results["Joint Angle"][index]) + '_' + str(axis)]))
        sd_list.append(sd)
        zscores = (normal_dist_data['stroke_'+str(results["Joint Angle"][index]) + '_' + str(axis)] - mean) / sd
        zscore_list.append(zscores)

    results["Normative Mean"] = mean_list
    results["Normative SD"] = sd_list
    results["Stroke Z-Score"] = zscore_list

    return results



def get_trial_ROM(df, feature_dict, first_result):
    df["feature"] = df["markers"] + '_'+ df["axis"]

    features_list = [i+'_'+j for i, k in feature_dict.items() for j in k]
    relevant_df = df[df["feature"].isin(features_list)]
    trial_rom_list = []
    for i in range(len(relevant_df)):
        trial_rom_list.append(max([float(k) for k in relevant_df.iloc[i,4:-1]]) - min([float(k) for k in relevant_df.iloc[i,4:-1]]))
        # print(trial_ROM)
    # test.iloc[:,4:-1]
    relevant_df["trial ROM"] = trial_rom_list
    average_list = collections.defaultdict()
    for feature in features_list:
        average_list[feature] = mean(relevant_df["trial ROM"][relevant_df["feature"] == feature])
    resorted_average_list = []
    for i in range(len(first_result)):
        if first_result["Feature"][i] == "Flexion/Extension":
            axis = "X"
        if first_result["Feature"][i] == "Abduction/Adduction":
            axis = "Y"
        if first_result["Feature"][i] == "External/Internal Rotation":
            axis = "Z"
        resorted_average_list.append(average_list[first_result["Joint Angle"][i] + '_' + axis])
    first_result["Trial ROM"] = resorted_average_list
    return relevant_df, features_list, first_result

def get_significance_vector(result_table, feature_dict, analyse_dict, ROM_dict):
    significancy_list = []
    rom_list = []
    total_calculation = []
    result_table = result_table.reset_index()
    for index in range(len(result_table)):
        for each_axis in feature_dict[result_table["Joint Angle"][index]]:
            for i, k in analyse_dict.items():
                if result_table["Joint Angle"][index] in k:
                    rangeOfMotion = tuple((i, each_axis))
                    break
        rom_list.append(ROM_dict[rangeOfMotion])
        significancy = (result_table["Trial ROM"][index]) / (int(ROM_dict[rangeOfMotion][1]) - int(ROM_dict[rangeOfMotion][0]))
        significancy_list.append(float(significancy))
    # print(significancy_list)
    result_table["ROM"] = rom_list
    result_table["weight"] = significancy_list

    
    return result_table

In [47]:
second_result_towel_vert_R = upper_threshold(result_towel_vert_R)
second_result_towel_hori_R = upper_threshold(result_towel_hori_R)
second_result_grasp_R = upper_threshold(result_grasp_R)
second_result_lateral_R = upper_threshold(result_lateral_R)

second_result_mouth_R = upper_threshold(result_mouth_R)

second_result_key_R = upper_threshold(result_key_R)

In [48]:
feature_dict_towel_vert_R = get_feature_dict(analyse_dict_R, TOWEL_vertFold)
relevant_towel_vert_R_df, towel_vert_R_feature_list, second_result_towel_vert_R = get_trial_ROM(towelvert_R_df, feature_dict_towel_vert_R, result_towel_vert_R)
third_result_towel_vert_R = get_significance_vector(second_result_towel_vert_R, feature_dict_towel_vert_R, analyse_dict_R, ROM_dict)

feature_dict_towel_hori_R = get_feature_dict(analyse_dict_R, TOWEL_horiFold)
relevant_towel_hori_R_df, towel_hori_R_feature_list, second_result_towel_hori_R = get_trial_ROM(towelhori_R_df, feature_dict_towel_hori_R, result_towel_hori_R)
third_result_towel_hori_R = get_significance_vector(second_result_towel_hori_R, feature_dict_towel_hori_R, analyse_dict_R, ROM_dict)

feature_dict_grasp_R = get_feature_dict(analyse_dict_R, GRASP)
relevant_grasp_R_df, grasp_R_feature_list, second_result_grasp_R = get_trial_ROM(grasp_R_df, feature_dict_grasp_R, result_grasp_R)
third_result_grasp_R = get_significance_vector(second_result_grasp_R, feature_dict_grasp_R, analyse_dict_R, ROM_dict)

feature_dict_lateral_R = get_feature_dict(analyse_dict_R, LATERAL)
relevant_lateral_R_df, lateral_R_feature_list, second_result_lateral_R = get_trial_ROM(lateral_R_df, feature_dict_lateral_R, result_lateral_R)
third_result_lateral_R = get_significance_vector(second_result_lateral_R, feature_dict_lateral_R, analyse_dict_R, ROM_dict)

feature_dict_mouth_R = get_feature_dict(analyse_dict_R, MOUTH)
relevant_mouth_R_df, mouth_R_feature_list, second_result_mouth_R = get_trial_ROM(mouth_R_df, feature_dict_mouth_R, result_mouth_R)
third_result_mouth_R = get_significance_vector(second_result_mouth_R, feature_dict_mouth_R, analyse_dict_R, ROM_dict)

feature_dict_key_R = get_feature_dict(analyse_dict_R, KEY)
relevant_key_R_df, key_R_feature_list, second_result_key_R = get_trial_ROM(key_R_df, feature_dict_key_R, result_key_R)
third_result_key_R = get_significance_vector(second_result_key_R, feature_dict_key_R, analyse_dict_R, ROM_dict)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7352\2130486487.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df["trial ROM"] = trial_rom_list
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7352\2130486487.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df["trial ROM"] = trial_rom_list
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7352\2130486487.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [55]:
fourth_result_towel_vert_R = get_norm_dist(third_result_towel_vert_R, norm_dist_towel_vert_R)
fourth_result_towel_hori_R = get_norm_dist(third_result_towel_hori_R, norm_dist_towel_hori_R)
fourth_result_grasp_R = get_norm_dist(third_result_grasp_R, norm_dist_grasp_R)
fourth_result_lateral_R = get_norm_dist(third_result_lateral_R, norm_dist_lateral_R)

fourth_result_mouth_R = get_norm_dist(third_result_mouth_R, norm_dist_mouth_R)

fourth_result_key_R = get_norm_dist(third_result_key_R, norm_dist_key_R)

In [68]:
def total_score(towel_vert_result, towel_hori_result, grasp_result, lateral_result, mouth_result, key_result):
    final_score_dict = collections.defaultdict() 
    final_score_towel_vert = sum(towel_vert_result["weight"] * towel_vert_result["Stroke Z-Score"])
    final_score_dict["Moving Towel"] = final_score_towel_vert
    final_score_towel_hori = sum(towel_hori_result["weight"] * towel_hori_result["Stroke Z-Score"])
    final_score_dict["Picking Block"] = final_score_towel_hori
    final_score_grasp = sum(grasp_result["weight"] * grasp_result["Stroke Z-Score"])
    final_score_dict["Shelving Items"] = final_score_grasp
    final_score_lateral = sum(lateral_result["weight"] * lateral_result["Stroke Z-Score"])
    final_score_dict["Scanning Goods"] = final_score_lateral
    final_score_mouth = sum(mouth_result["weight"] * mouth_result["Stroke Z-Score"])
    final_score_dict["Eating"] = final_score_mouth
    final_score_key = sum(key_result["weight"] * key_result["Stroke Z-Score"])
    final_score_dict["Pouring Drinks"] = final_score_key

    descending_final_score_dict = OrderedDict(sorted(final_score_dict.items(), key=lambda item: item[1], reverse=False))
    return descending_final_score_dict


In [80]:
final_results = total_score(fourth_result_towel_vert_R, fourth_result_towel_hori_R, fourth_result_grasp_R, fourth_result_lateral_R, fourth_result_mouth_R, fourth_result_mouth_R)

In [81]:
final_results

OrderedDict([('Picking Block', 0.6012477803994194),
             ('Moving Towel', 1.8761610189646178),
             ('Shelving Items', 3.0219394522899714),
             ('Scanning Goods', 3.800415376964108),
             ('Eating', 4.2052873937803446),
             ('Pouring Drinks', 4.2052873937803446)])

In [9]:
forpickle = [towelvert_R_df, result_towel_vert_R, norm_dist_towel_vert_R, 

towelhori_R_df, result_towel_hori_R, norm_dist_towel_hori_R,

grasp_R_df, result_grasp_R, norm_dist_grasp_R,

lateral_R_df, result_lateral_R, norm_dist_lateral_R,

mouth_R_df, result_mouth_R, norm_dist_mouth_R ,

key_R_df, result_key_R, norm_dist_key_R ]

In [78]:
pickle_out = open(r"C:\Users\Administrator\Documents\gitHub\BCI\data\result_v2.pickle","wb")
pickle.dump(forpickle, pickle_out)
pickle_out.close()

In [13]:
file = open('C:\Users\Administrator\Documents\gitHub\BCI\data\result_v2.pickle', 'rb')

# dump information to that file
data = pickle.load(file)

# close the file
file.close()




NameError: name 'towelvert_R_df' is not defined

In [6]:
load_data = collections.defaultdict()

In [29]:
for i in range(len(forpickle)):
    if forpickle[i][0:4] == 'norm':
        load_data[forpickle[i]] = data[i]
    else:
        load_data[forpickle[i]] = pd.DataFrame(data[i])


In [42]:
towelvert_R_df = load_data['towelvert_R_df']
towelhori_R_df = load_data['towelhori_R_df']
grasp_R_df = load_data['grasp_R_df']
lateral_R_df = load_data['lateral_R_df']
mouth_R_df = load_data['mouth_R_df']
key_R_df = load_data['key_R_df']

In [39]:
result_towel_vert_R = load_data['result_towel_vert_R']
result_towel_hori_R = load_data['result_towel_hori_R']
result_grasp_R = load_data['result_grasp_R']
result_lateral_R = load_data['result_lateral_R']
result_mouth_R = load_data['result_mouth_R']
result_key_R = load_data['result_key_R']

In [44]:
norm_dist_towel_vert_R = load_data['norm_dist_towel_vert_R']
norm_dist_towel_hori_R = load_data['norm_dist_towel_hori_R']
norm_dist_grasp_R = load_data['norm_dist_grasp_R']
norm_dist_lateral_R = load_data['norm_dist_lateral_R']
norm_dist_mouth_R = load_data['norm_dist_mouth_R']
norm_dist_key_R = load_data['norm_dist_key_R']

In [62]:
second_result_towel_vert_R = upper_threshold(result_towel_vert_R)
second_result_towel_hori_R = upper_threshold(result_towel_hori_R)
second_result_grasp_R = upper_threshold(result_grasp_R)
second_result_lateral_R = upper_threshold(result_lateral_R)

second_result_mouth_R = upper_threshold(result_mouth_R)

second_result_key_R = upper_threshold(result_key_R)

In [63]:
feature_dict_towel_vert_R = get_feature_dict(analyse_dict_R, TOWEL_vertFold)
relevant_towel_vert_R_df, towel_vert_R_feature_list, second_result_towel_vert_R = get_trial_ROM(towelvert_R_df, feature_dict_towel_vert_R, result_towel_vert_R)
third_result_towel_vert_R = get_significance_vector(second_result_towel_vert_R, feature_dict_towel_vert_R, analyse_dict_R, ROM_dict)

feature_dict_towel_hori_R = get_feature_dict(analyse_dict_R, TOWEL_horiFold)
relevant_towel_hori_R_df, towel_hori_R_feature_list, second_result_towel_hori_R = get_trial_ROM(towelhori_R_df, feature_dict_towel_hori_R, result_towel_hori_R)
third_result_towel_hori_R = get_significance_vector(second_result_towel_hori_R, feature_dict_towel_hori_R, analyse_dict_R, ROM_dict)

feature_dict_grasp_R = get_feature_dict(analyse_dict_R, GRASP)
relevant_grasp_R_df, grasp_R_feature_list, second_result_grasp_R = get_trial_ROM(grasp_R_df, feature_dict_grasp_R, result_grasp_R)
third_result_grasp_R = get_significance_vector(second_result_grasp_R, feature_dict_grasp_R, analyse_dict_R, ROM_dict)

feature_dict_lateral_R = get_feature_dict(analyse_dict_R, LATERAL)
relevant_lateral_R_df, lateral_R_feature_list, second_result_lateral_R = get_trial_ROM(lateral_R_df, feature_dict_lateral_R, result_lateral_R)
third_result_lateral_R = get_significance_vector(second_result_lateral_R, feature_dict_lateral_R, analyse_dict_R, ROM_dict)

feature_dict_mouth_R = get_feature_dict(analyse_dict_R, MOUTH)
relevant_mouth_R_df, mouth_R_feature_list, second_result_mouth_R = get_trial_ROM(mouth_R_df, feature_dict_mouth_R, result_mouth_R)
third_result_mouth_R = get_significance_vector(second_result_mouth_R, feature_dict_mouth_R, analyse_dict_R, ROM_dict)

feature_dict_key_R = get_feature_dict(analyse_dict_R, KEY)
relevant_key_R_df, key_R_feature_list, second_result_key_R = get_trial_ROM(key_R_df, feature_dict_key_R, result_key_R)
third_result_key_R = get_significance_vector(second_result_key_R, feature_dict_key_R, analyse_dict_R, ROM_dict)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11612\3148919282.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df["trial ROM"] = trial_rom_list
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11612\3148919282.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df["trial ROM"] = trial_rom_list
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11612\3148919282.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [64]:
fourth_result_towel_vert_R = get_norm_dist(third_result_towel_vert_R, norm_dist_towel_vert_R)
fourth_result_towel_hori_R = get_norm_dist(third_result_towel_hori_R, norm_dist_towel_hori_R)
fourth_result_grasp_R = get_norm_dist(third_result_grasp_R, norm_dist_grasp_R)
fourth_result_lateral_R = get_norm_dist(third_result_lateral_R, norm_dist_lateral_R)

fourth_result_mouth_R = get_norm_dist(third_result_mouth_R, norm_dist_mouth_R)

fourth_result_key_R = get_norm_dist(third_result_key_R, norm_dist_key_R)

In [69]:
final_results = total_score(fourth_result_towel_vert_R, fourth_result_towel_hori_R, fourth_result_grasp_R, fourth_result_lateral_R, fourth_result_mouth_R, fourth_result_mouth_R)
final_results

OrderedDict([('Picking Block', 0.6217166928892461),
             ('Moving Towel', 1.8761610189646178),
             ('Shelving Items', 3.0219394522899714),
             ('Scanning Goods', 3.800415376964108),
             ('Eating', 4.2052873937803446),
             ('Pouring Drinks', 4.2052873937803446)])

Picking Block', 0.6012477803994194),
             ('Moving Towel', 1.8761610189646178),
             ('Shelving Items', 3.0219394522899714),
             ('Scanning Goods', 3.800415376964108),
             ('Eating', 4.2052873937803446),
             ('Pouring Drinks', 4.2052873937803446)]

#NOTE:
dont drop the joint angle, only drop if everything passes the normative band (the whole action)

# Picking Block

##### 1. Upper Threshold (Meet Normative Band)

In [70]:
second_result_towel_hori_R

,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Drop (Upper threshold),Trial ROM
0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,107.055635,"(62.3163363631262, 167.12202091012347)",1,31.558990
1,R_Shoulder_JOINT_ANGLE,Abduction/Adduction,439.641522,"(31.35063890485658, 167.18235494014766)",0,26.232062
2,R_Elbow_JOINT_ANGLE,Flexion/Extension,364.85691,"(71.99701359605321, 143.67835038012583)",0,30.163941
3,R_Wrist_JOINT_ANGLE,Flexion/Extension,231.608046,"(101.86383965250542, 253.6300973606683)",1,15.575298


##### 2. Lower Threshold

In [71]:
fourth_result_towel_hori_R

,index,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Drop (Upper threshold),Trial ROM,ROM,weight,Normative Mean,Normative SD,Stroke Z-Score
0,0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,107.055635,"(62.3163363631262, 167.12202091012347)",1,31.558990,"(-180, 50)",0.137213,114.719179,184.482523,-0.041541
1,1,R_Shoulder_JOINT_ANGLE,Abduction/Adduction,439.641522,"(31.35063890485658, 167.18235494014766)",0,26.232062,"(-180, 50)",0.114052,99.266497,239.095596,1.423594
2,2,R_Elbow_JOINT_ANGLE,Flexion/Extension,364.85691,"(71.99701359605321, 143.67835038012583)",0,30.163941,"(0, 140)",0.215457,107.837682,126.175922,2.036991
3,3,R_Wrist_JOINT_ANGLE,Flexion/Extension,231.608046,"(101.86383965250542, 253.6300973606683)",1,15.575298,"(-60, 60)",0.129794,177.746969,267.144117,0.201618


# Moving Towel

##### 1. Upper Threshold

In [51]:
second_result_towel_vert_R

,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Trial ROM,Drop (Upper threshold)
0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,220.433361,"(33.158478750659455, 142.74346862990348)",54.503035,0
1,R_Elbow_JOINT_ANGLE,Flexion/Extension,579.796379,"(42.710250789875786, 106.52498220188617)",55.035373,0
2,R_Wrist_JOINT_ANGLE,Flexion/Extension,271.090325,"(98.47583020459722, 233.86220745854393)",16.386324,0


##### 2. Lower Threshold

In [52]:
fourth_result_towel_vert_R

,index,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Trial ROM,Drop (Upper threshold),ROM,weight,Normative Mean,Normative SD,Stroke Z-Score
0,0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,220.433361,"(33.158478750659455, 142.74346862990348)",54.503035,0,"(-50, 180)",0.236970,87.950974,204.683083,0.647256
1,1,R_Elbow_JOINT_ANGLE,Flexion/Extension,579.796379,"(42.710250789875786, 106.52498220188617)",55.035373,0,"(0, 140)",0.393110,74.617616,119.193295,4.238315
2,2,R_Wrist_JOINT_ANGLE,Flexion/Extension,271.090325,"(98.47583020459722, 233.86220745854393)",16.386324,0,"(-60, 60)",0.136553,166.169019,252.874971,0.414914


# Shelving Items

##### 1. Upper Threshold

In [53]:
second_result_grasp_R

,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Normative Mean,Normative SD,Stroke Z-Score,Drop (Upper threshold),Trial ROM
0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,457.070002,"(70.266951205847, 125.72046491274912)",97.993708,103.576194,[3.4667840180877207],0,76.790746
1,R_Elbow_JOINT_ANGLE,Flexion/Extension,641.313831,"(74.91468883931682, 147.72354122536566)",111.319115,135.992533,[3.897233930914154],0,57.964819
2,R_Wrist_JOINT_ANGLE,Flexion/Extension,552.890893,"(216.60583338658932, 300.6026358140475)",258.604235,156.889411,[1.875758577818198],0,16.050180


##### 2. Lower Threshold

In [54]:
fourth_result_grasp_R

,index,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Normative Mean,Normative SD,Stroke Z-Score,Drop (Upper threshold),Trial ROM,ROM,weight
0,0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,457.070002,"(70.266951205847, 125.72046491274912)",97.993708,103.576194,3.466784,0,76.790746,"(-50, 180)",0.333873
1,1,R_Elbow_JOINT_ANGLE,Flexion/Extension,641.313831,"(74.91468883931682, 147.72354122536566)",111.319115,135.992533,3.897234,0,57.964819,"(0, 140)",0.414034
2,2,R_Wrist_JOINT_ANGLE,Flexion/Extension,552.890893,"(216.60583338658932, 300.6026358140475)",258.604235,156.889411,1.875759,0,16.050180,"(-60, 60)",0.133752


# Scanning Goods

##### 1. Upper Threshold

In [55]:
second_result_lateral_R

,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Drop (Upper threshold),Trial ROM
0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,44.879062,"(61.20589865739277, 91.14285764888739)",0,41.723978
1,R_Shoulder_JOINT_ANGLE,Abduction/Adduction,436.644106,"(44.472051566965256, 60.73347004120222)",0,22.513315
2,R_Elbow_JOINT_ANGLE,Flexion/Extension,534.324116,"(60.51477651128577, 89.89703522365977)",0,45.761240


##### 2. Lower Threshold

In [56]:
fourth_result_lateral_R

,index,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Drop (Upper threshold),Trial ROM,ROM,weight,Normative Mean,Normative SD,Stroke Z-Score
0,0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,44.879062,"(61.20589865739277, 91.14285764888739)",0,41.723978,"(-180, 50)",0.181409,76.174378,56.949367,-0.549529
1,1,R_Shoulder_JOINT_ANGLE,Abduction/Adduction,436.644106,"(44.472051566965256, 60.73347004120222)",0,22.513315,"(-180, 50)",0.097884,52.602761,30.934254,12.414760
2,2,R_Elbow_JOINT_ANGLE,Flexion/Extension,534.324116,"(60.51477651128577, 89.89703522365977)",0,45.761240,"(0, 140)",0.326866,75.205906,55.894155,8.214065


# Eating

##### 1. Upper Threshold

In [57]:
second_result_mouth_R

,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Drop (Upper threshold),Trial ROM
0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,38.070146,"(68.79013449446767, 97.42903220412556)",0,31.939363
1,R_Shoulder_JOINT_ANGLE,Abduction/Adduction,344.624732,"(59.009771022994535, 85.61156437336031)",0,10.191333
2,R_Elbow_JOINT_ANGLE,Flexion/Extension,318.215491,"(59.21025536982168, 90.23094976886892)",0,70.323472
3,R_Elbow_JOINT_ANGLE,External/Internal Rotation,477.994687,"(111.44152994133981, 164.04127412488305)",0,91.730517
4,R_Wrist_JOINT_ANGLE,Flexion/Extension,467.9359,"(94.60162911300094, 185.54440358069652)",0,15.338998


##### 2. Lower Threshold

In [58]:
fourth_result_mouth_R

,index,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Drop (Upper threshold),Trial ROM,ROM,weight,Normative Mean,Normative SD,Stroke Z-Score
0,0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,38.070146,"(68.79013449446767, 97.42903220412556)",0,31.939363,"(-180, 50)",0.138867,83.109583,53.491796,-0.841988
1,1,R_Shoulder_JOINT_ANGLE,Abduction/Adduction,344.624732,"(59.009771022994535, 85.61156437336031)",0,10.191333,"(-180, 50)",0.044310,72.310668,49.686888,5.480602
2,2,R_Elbow_JOINT_ANGLE,Flexion/Extension,318.215491,"(59.21025536982168, 90.23094976886892)",0,70.323472,"(-80, 80)",0.439522,74.720603,57.940521,4.202497
3,3,R_Elbow_JOINT_ANGLE,External/Internal Rotation,477.994687,"(111.44152994133981, 164.04127412488305)",0,91.730517,"(-80, 80)",0.573316,137.741402,98.245917,3.463282
4,4,R_Wrist_JOINT_ANGLE,Flexion/Extension,467.9359,"(94.60162911300094, 185.54440358069652)",0,15.338998,"(-60, 60)",0.127825,140.073016,169.863113,1.930159


# Pouring Drinks

##### 1. Upper Threshold

In [59]:
second_result_key_R

,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Drop (Upper threshold),Trial ROM
0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,639.597818,"(81.2358286336372, 136.03850675196404)",0,2.016744
1,R_Shoulder_JOINT_ANGLE,Abduction/Adduction,513.700996,"(41.62002552688662, 85.62888242238256)",0,5.599722
2,R_Elbow_JOINT_ANGLE,Flexion/Extension,542.169139,"(157.77947671729547, 238.6326882893181)",0,9.323468
3,R_Elbow_JOINT_ANGLE,External/Internal Rotation,1056.260784,"(77.06671051387644, 319.53136129515053)",0,128.085953


##### 2. Lower Threshold

In [60]:
fourth_result_key_R

,index,Joint Angle,Feature,Stroke Patient Score,Normative Confidence Interval,Drop (Upper threshold),Trial ROM,ROM,weight,Normative Mean,Normative SD,Stroke Z-Score
0,0,R_Shoulder_JOINT_ANGLE,Flexion/Extension,639.597818,"(81.2358286336372, 136.03850675196404)",0,2.016744,"(-180, 50)",0.008768,108.637168,95.447720,5.562843
1,1,R_Shoulder_JOINT_ANGLE,Abduction/Adduction,513.700996,"(41.62002552688662, 85.62888242238256)",0,5.599722,"(-180, 50)",0.024347,63.624454,76.648536,5.871952
2,2,R_Elbow_JOINT_ANGLE,Flexion/Extension,542.169139,"(157.77947671729547, 238.6326882893181)",0,9.323468,"(-80, 80)",0.058272,198.206083,140.818933,2.442591
3,3,R_Elbow_JOINT_ANGLE,External/Internal Rotation,1056.260784,"(77.06671051387644, 319.53136129515053)",0,128.085953,"(-80, 80)",0.800537,198.299036,422.291369,2.031682
